In [2]:
!pip install streamlit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 31.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 54.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.9/82.9 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 3.9 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0


In [5]:
!pip install rembg

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 66.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 5.3 MB/s eta 0:00:00


In [6]:
%%writefile app.py

import io
import zipfile
from pathlib import Path
import streamlit as st
from PIL import Image, ImageEnhance
from rembg import remove
import uuid
from io import BytesIO

MAX_FILES = 5
ALLOWED_TYPES = ["png", "jpg", "jpeg"]

def setup_page():
    """Sets up the Streamlit page configuration."""
    st.set_page_config(page_title="Background Remover", page_icon="✂️")
    hide_streamlit_style()

def hide_streamlit_style():
    """Hides default Streamlit styling."""
    st.markdown(
        "<style>footer {visibility: hidden;} #MainMenu {visibility: hidden;}</style>",
        unsafe_allow_html=True,
    )

def initialize_session():
    """Initializes a unique session ID."""
    if "uploader_key" not in st.session_state:
        st.session_state["uploader_key"] = str(uuid.uuid4())

def display_ui():
    """Displays the user interface for file upload and returns uploaded files."""
    st.sidebar.markdown("## Image Background Remover")

    uploaded_files = st.sidebar.file_uploader(
        "Choose images (Drag-and-Drop supported)",
        type=ALLOWED_TYPES,
        accept_multiple_files=True,
        key=st.session_state.get("uploader_key", "file_uploader"),
    )

    # Output file type selection
    output_format = st.sidebar.selectbox("Select output format", ["PNG", "JPG", "JPEG"])

    # Quality adjustment slider for JPG output
    if output_format in ["JPG", "JPEG"]:
        quality = st.sidebar.slider("Adjust Quality (JPG/JPEG)", 1, 100, 85)
    else:
        quality = 100  # Default for PNG

    # Image filters
    filter_option = st.sidebar.selectbox(
        "Apply Filters to the Result", ["None", "Grayscale", "Blur"]
    )

    display_footer()
    return uploaded_files, output_format, quality, filter_option

def display_footer():
    """Displays a custom footer."""
    footer = """<div style="position: fixed; bottom: 0; left: 20px;">
                <p>Developed with ❤ by <a href="https://github.com/ArraAkshaya/BackgroundRemover" target="_blank">@ArraAkshaya</a></p>
                </div>"""
    st.sidebar.markdown(footer, unsafe_allow_html=True)

def process_and_display_images(uploaded_files, output_format, quality, filter_option):
    """Processes the uploaded files and displays the original and result images."""
    if not uploaded_files:
        st.warning("Please upload an image.")
        return

    if not st.sidebar.button("Remove Background"):
        return

    if len(uploaded_files) > MAX_FILES:
        st.warning(f"Maximum {MAX_FILES} files will be processed.")
        uploaded_files = uploaded_files[:MAX_FILES]

    results = []

    with st.spinner("Removing backgrounds..."):
        for uploaded_file in uploaded_files:
            original_image = Image.open(uploaded_file).convert("RGBA")
            result_image = remove_background(uploaded_file.getvalue())
            # Apply filters
            result_image = apply_filters(result_image, filter_option)
            results.append((original_image, result_image, uploaded_file.name))

    for original, result, name in results:
        col1, col2 = st.columns(2)
        with col1:
            st.image(original, caption="Original")
        with col2:
            st.image(result, caption="Result")

    if len(results) > 1:
        download_zip(results, output_format, quality)
    else:
        download_result(results[0], output_format, quality)

def remove_background(image_bytes):
    """Removes the background from an image."""
    result = remove(image_bytes)
    return Image.open(io.BytesIO(result)).convert("RGBA")

def apply_filters(image, filter_option):
    """Applies the selected filter to the image."""
    if filter_option == "Grayscale":
        image = image.convert("L").convert("RGBA")
    elif filter_option == "Blur":
        image = image.filter(ImageFilter.GaussianBlur(2))
    return image

def img_to_bytes(img, format, quality):
    """Converts an Image object to bytes."""
    buf = io.BytesIO()
    img.save(buf, format=format, quality=quality)
    return buf.getvalue()

def download_result(image, output_format, quality):
    """Allows the user to download the result image."""
    _, result, name = image
    st.download_button(
        label="Download Result",
        data=img_to_bytes(result, format=output_format.upper(), quality=quality),
        file_name=f"{Path(name).stem}_nobg.{output_format.lower()}",
        mime=f"image/{output_format.lower()}",
    )

def download_zip(results, output_format, quality):
    """Creates and downloads a zip file of all processed images."""
    zip_buffer = BytesIO()
    with zipfile.ZipFile(zip_buffer, "w") as zf:
        for _, result, name in results:
            img_data = img_to_bytes(result, format=output_format.upper(), quality=quality)
            zf.writestr(f"{Path(name).stem}_nobg.{output_format.lower()}", img_data)

    st.download_button(
        label="Download All as ZIP",
        data=zip_buffer.getvalue(),
        file_name="processed_images.zip",
        mime="application/zip",
    )

if __name__ == "__main__":
    setup_page()
    initialize_session()
    uploaded_files, output_format, quality, filter_option = display_ui()
    process_and_display_images(uploaded_files, output_format, quality, filter_option)

Writing app.py


In [7]:
!npm install localtunnel


added 22 packages, and audited 23 packages in 2s

3 packages are looking for funding
  run `npm fund` for details

2 moderate severity vulnerabilities

To address all issues, run:
  npm audit fix

Run `npm audit` for details.


In [9]:
import urllib
print("Password/Enpoint IP for localtunnel is:",urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n"))

Password/Enpoint IP for localtunnel is: 35.192.18.112


In [10]:
!streamlit run app.py &>/content/logs.txt &

In [ ]:
!npx localtunnel --port 8501

your url is: https://violet-nails-wonder.loca.lt
